In [1]:
import json
import nltk # stemmer
import numpy as np
import pandas as pd
import patsy
import re
import sklearn as skl
import string
import time

# Preprocess data

In [68]:
train = pd.read_json("data/music_200.json", lines=True)
test = pd.read_json("data/music_test_200.json", lines=True)
train.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [69]:
train = train.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
test = test.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
train.head()

,overall,reviewText
0,5,"Not much to write about here, but it does exac..."
1,5,The product does exactly as it should and is q...
2,5,The primary job of this device is to block the...
3,5,Nice windscreen protects my MXL mic and preven...
4,5,This pop filter is great. It looks and perform...


In [121]:
train2 = train.assign(good = lambda g: g.overall >= 5)
trainFinal = train.assign(good = train2['good'].apply(lambda g: 1 if g else 0))

In [122]:
def rm_stopwords_punctuation(text):
    text = text.lower()
    with open("stopwords.json") as stopword_file:
        stopwords = json.load(stopword_file)
        for word in stopwords:
            if word in text:
                # replace only complete words ('\b' is a word boundary)
                text = re.sub(r"\b{}\b".format(word), "", text)
    # remove punctuation
    for char in string.punctuation:
        text = text.replace(char, "")
    text = re.sub(r"\b[a-z]\b", "", text)
    # remove whitespace
    for char in string.punctuation:
        text = text.replace(char, "")
    text = ' '.join(text.split(None))
    return text

In [123]:
def stem(text):
    stemmer = nltk.stem.porter.PorterStemmer()
    # stem each word individually, and concatenate
    text = ' '.join([stemmer.stem(word) for word in text.split(None)])
#     text = [stemmer.stem(word) for word in text.split(None)]
    return text

In [124]:
def process_text(text):
    text = rm_stopwords_punctuation(text)
    text = stem(text)
    return text

In [125]:
trainNoProcess = trainFinal.copy()
trainFinal['reviewText'] = trainFinal['reviewText'].apply(lambda t: process_text(t))

In [75]:
test2 = test.assign(good = lambda g: g.overall >= 4)
testFinal = test.assign(good = test2['good'].apply(lambda g: 1 if g else 0))
testNoProcess = testFinal.copy()
testFinal['reviewText'] = testFinal['reviewText'].apply(lambda t: process_text(t))

In [126]:
y, X = patsy.dmatrices("good ~ reviewText", trainFinal, return_type="dataframe")
y_test, X_test = patsy.dmatrices("good ~ reviewText", testFinal, return_type="dataframe")
y_noP, X_noP = patsy.dmatrices("good ~ reviewText", trainNoProcess, return_type="dataframe")
y_test_noP, X_test_noP = patsy.dmatrices("good ~ reviewText", testNoProcess, return_type="dataframe")

In [57]:
print(y)
# print(X)
print(len(y), len(X))

     good
0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     0.0
8     1.0
9     1.0
10    1.0
11    1.0
12    0.0
13    1.0
14    1.0
15    0.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
..    ...
170   1.0
171   1.0
172   1.0
173   1.0
174   1.0
175   1.0
176   1.0
177   1.0
178   1.0
179   0.0
180   1.0
181   1.0
182   1.0
183   1.0
184   1.0
185   1.0
186   1.0
187   1.0
188   1.0
189   1.0
190   1.0
191   1.0
192   1.0
193   1.0
194   1.0
195   1.0
196   1.0
197   1.0
198   1.0
199   1.0

[200 rows x 1 columns]
200 200


In [138]:
start_time = time.time()
# vg_test = pd.read_json("data/reviews_Video_Games_5.json", lines=True)
vg_test = pd.read_json("data/reviews_Musical_Instruments_5.json", lines=True)
vg_test = vg_test.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
elapsed_time = time.time() - start_time
vg_test.describe()
print(elapsed_time)


0.7640759944915771


In [139]:
start_time = time.time()
vg_test2 = vg_test.assign(good = lambda g: g.overall >= 5)
vg_testFinal = vg_test.assign(good = vg_test2['good'].apply(lambda g: 1 if g else 0))
vg_test_noP = vg_testFinal.copy()
# vg_testFinal['reviewText'] = vg_testFinal['reviewText'].apply(lambda t: process_text(t))
# y_vg_test, X_vg_test = patsy.dmatrices("good ~ reviewText", testNoProcess, return_type="dataframe")
elapsed_time = time.time() - start_time
print(elapsed_time)

0.023331880569458008


In [140]:
start_time = time.time()
print(len(vg_test_noP))
y_vg_test_noP, X_vg_test_noP = patsy.dmatrices("good ~ reviewText", vg_test_noP, return_type="dataframe")
elapsed_time = time.time() - start_time
print(len(y_vg_test_noP), len(X_vg_test_noP))
print(elapsed_time)

10261
10261 10261
19.28083300590515


# Logistic Regression

In [ ]:
logRegrModel = skl.linear_model.LogisticRegression()
logRegrModel = logRegrModel.fit(X, y['good'])

In [ ]:
logRegrModel.score(X, y['good'])

In [ ]:
logRegrModel.score(X_test, y_test['good'])

In [21]:
logRegrModel_noP = skl.linear_model.LogisticRegression()
logRegrModel_noP = logRegrModel.fit(X_noP, y_noP['good'])

In [22]:
logRegrModel_noP.score(X_noP, y_noP['good'])

0.92000000000000004

In [23]:
logRegrModel.score(X_test_noP, y_test_noP['good'])

0.89000000000000001

In [44]:
y.mean()

good    0.92
dtype: float64

In [45]:
# logRegrModel.score(X_vg_test, y_vg_test['good'])
logRegrModel_noP.fit_transform(X_vg_test_noP, np.ravel(y_vg_test_noP))
# logRegrModel_noP.score(X_vg_test_noP, y_vg_test_noP['good'])
# logRegrModel_noP.score(X_noP, y_noP['good'])
y_vg_test_noP.mean()

/Users/et/Documents/Code/virtualenv/py_standard/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


good    0.879252
dtype: float64

In [51]:
pd.DataFrame(list(zip(X_vg_test_noP.columns, np.transpose(logRegrModel_noP.coef_))))

,0,1
0,Intercept,[1.00926676996]
1,"reviewText[T.""Oh its a crybaby"", ""I have a cry...",[-0.776221429155]
2,reviewText[T.$49 for a $199 microphone at Amaz...,[0.10670482864]
3,reviewText[T.&#34;Nice for the price&#34; ... ...,[-0.776221429155]
4,reviewText[T.&#34;Quality of this banjo exceed...,[0.10670482864]
5,reviewText[T.(This review is legit)I bought th...,[0.10670482864]
6,reviewText[T.(star rating based on price) grea...,[0.10670482864]
7,reviewText[T.* SHIPPING. The Epiphone comes w...,[0.10670482864]
8,reviewText[T.***Update*** After heavy play for...,[0.10670482864]
9,reviewText[T.**So after reading a couple comme...,[0.10670482864]


In [28]:
big_noP = skl.linear_model.LogisticRegression()
big_noP = logRegrModel.fit(X_vg_test_noP, y_vg_test_noP['good'])

In [29]:
big_noP.score(X_vg_test_noP, y_vg_test_noP['good'])

0.87925153493811514

# test stuff

In [128]:
# cv = skl.feature_extraction.text.CountVectorizer(list(trainFinal['reviewText']))
cv = skl.feature_extraction.text.CountVectorizer()

In [141]:
cv

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [142]:
trainFinal['reviewText'][199]

'use mic snare drum sometim guitar cabinet work well mic durabl use studio noth els avail work fine long run sound live will keep mic'

In [143]:
# counts = cv.fit_transform(list(trainFinal['reviewText']))
counts = cv.fit_transform(list(vg_test_noP['reviewText']))

In [144]:
counts.shape

(10261, 20551)

In [145]:
tf_transformer = skl.feature_extraction.text.TfidfTransformer(use_idf=False).fit(counts)

In [146]:
train_tf = tf_transformer.transform(counts)
train_tf

<10261x20551 sparse matrix of type '<class 'numpy.float64'>'
	with 595985 stored elements in Compressed Sparse Row format>

In [147]:
tfidf_transformer = skl.feature_extraction.text.TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(counts)
train_tfidf

<10261x20551 sparse matrix of type '<class 'numpy.float64'>'
	with 595985 stored elements in Compressed Sparse Row format>

In [148]:
sklmodel = skl.linear_model.LogisticRegression()
sklmodel = sklmodel.fit(train_tfidf, y_vg_test_noP['good'])

In [149]:
sklmodel.score(train_tfidf, y_vg_test_noP['good'])

0.82418867556768349

In [150]:
y.mean()

good    0.67
dtype: float64